# Edge Trees Analysis

## Import settings

In [2]:
import json
from pathlib import Path
import os

#settings_file = r"Q:\Projekt\Data_2024\styrfiler\settings_SEKNNO.json"
#settings_file = r"Q:\Projekt\Data_2024\styrfiler\settings_SEVPLI.json"
#settings_file = r"C:\SVK_utveckling\settings_SEVPLI.json"
settings_file = r"C:\Users\SE1K4H\Desktop\SVK-Analys-Filer\settings_SEKNNO.json" # Working on Elsas's computer

try:
    if not os.path.exists(settings_file):
        raise FileNotFoundError(f"Could not find {settings_file}")

    with open(settings_file, 'r', encoding='utf-8') as file:
        settings = json.load(file)

    # TODO: check which ones are needed and not
    run_ID = settings["run_ID"]
    powerline_list = settings["powerline_list"]
    local_dir = settings["local_folder"]
    working_gdb_template = settings["working_gdb_template"]
    wires_gdb_template = settings["wires_gdb_template"]
    domains_folder = settings["domains_folder"]
    scandate_file = settings["scandate_file"]
    cvd_LEDNINGSGATA_path = os.path.join(domains_folder, "cvd_LEDNINGSGATA.txt")
    powerlines_folder = settings["powerlines_folder"]
    LG_polygons = settings["LG_polygons"]
    station_polygons = settings["station_polygons"]
    module_path = settings["modules"] 
    ogr2ogr_path = settings["ogr2ogr_path"]   
    proj_lib_path = settings["proj_lib_path"]
    gdal_data_path = settings["gdal_data_path"]
    DEFAULT_DB_NAME = settings["default_db_name"]
    DB_NAME = settings["db_name"]
    USER = settings["db_user"]
    PASSWORD = settings["db_password"]
    HOST = settings["host"]
    PORT = settings["port"] 

    if None in (run_ID, powerline_list, local_dir, working_gdb_template, wires_gdb_template, domains_folder, scandate_file, powerlines_folder, LG_polygons, station_polygons, module_path, ogr2ogr_path, proj_lib_path, gdal_data_path, DEFAULT_DB_NAME, DB_NAME, USER, PASSWORD, HOST, PORT):
        raise KeyError(f"One or more keys are missing in {settings_file}")
    
    working_gdb = os.path.join(local_dir, f"working_{run_ID}_PostGIS.gdb")
    
    print(f"Settings loaded successfully.")
except json.JSONDecodeError as e:
    print(f"Error: Invalid JSON format in {settings_file}")
except KeyError as e:
    print(f"Error: {e}")
except FileNotFoundError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"Error: Unexpected error: {e}")

Settings loaded successfully.


## Load PostGIS methods

In [13]:
import psycopg

# TODO can move these kind of methods to utils or similiar?
def connect_to_database(db_name):
    try:
        conn = psycopg.connect(dbname=db_name, user=USER, password=PASSWORD, host=HOST, port=PORT)
        conn.autocommit = True
        return conn
    except psycopg.Error as e:
        print(f"Error connecting to {db_name}: {e}")
        return None
    
def execute_query(conn, query, data=None, fetch=False):
    try:
        with conn.cursor() as cur:
            cur.execute(query, data or ())
            if fetch:
                return cur.fetchall()
    except psycopg.Error as e:
        print(f"Error executing query: {e}")

## Copy template gdb

In [4]:
import os
import shutil

if os.path.exists(working_gdb):
    print(f"{working_gdb} already exists")
else: 
    shutil.copytree(working_gdb_template, working_gdb)

C:/Users/SE1K4H/Desktop/SVK-Analys-Filer/pythonkörningar/test_postGIS\working_250201_PostGIS.gdb already exists


## Merge SKB text files for all blocks of a powerline

In [5]:
from pathlib import Path
import glob
import fileinput
import pandas as pd
import os

# TODO add exception handling.

def combine_blocks(row):
    LG = row["LG"]
    line = row["line"]
    line_dir = Path(powerlines_folder) / LG / f"line_{line}"
    
    block_dir = os.path.join(line_dir, "kantträd", "block")
    combined_blocks_path = os.path.join(line_dir, "kantträd", "SKB_raw.txt")
    
    # Merge files for all blocks into one
    merge_blocks(block_dir, "*.txt", combined_blocks_path)
    print(f"Successfully merged all edge trees blocks for {LG}/line_{line} into text file {combined_blocks_path}.")

def merge_blocks(src_dir, search_pattern, dst_file):
    blocks = glob.glob(os.path.join(src_dir, search_pattern))
    with open(dst_file, "w") as fh:
        input_lines = fileinput.input(blocks)
        fh.writelines(input_lines)

powerlines_df = pd.read_csv(powerline_list, sep="\t", header=0)
powerlines_df.apply(combine_blocks, axis=1)
print(f"Done with all power lines")

Successfully merged all edge trees blocks for LG001/line_1 into text file C:\Users\SE1K4H\Desktop\SVK-Analys-Filer\ledningar\LG001\line_1\kantträd\SKB_raw.txt.
Done with all power lines


## Add Edge Trees to PostGIS Tables 

In [6]:
import psycopg
from pathlib import Path
import pandas as pd
import os
from psycopg import sql #TODO update to and install psycopg, the latest version.

cur = None
conn = None

def create_SKB_XYmZ_table(row):
    LG = row["LG"]
    line = row["line"]
    table_name = f"{LG.lower()}_{line}_skb_xymz"
    line_dir = Path(powerlines_folder) / LG / f"line_{line}"
    SKB_dir = line_dir / "kantträd"
    SKB_file_in = os.path.join(SKB_dir, f"SKB_raw.txt")
    drop_table_if_exists_query = sql.SQL("DROP TABLE IF EXISTS {table_name}").format(table_name=sql.Identifier(table_name))
    create_query = sql.SQL("CREATE TABLE {table_name}(objectid SERIAL PRIMARY KEY, x DOUBLE PRECISION, y DOUBLE PRECISION, z FLOAT, dz FLOAT, mz FLOAT, shape geometry(POINTZ, 3006))").format(table_name=sql.Identifier(table_name)) #TODO potentially change to 5845

    try:
        # Step 1: Create table for powerline
        execute_query(conn_db, drop_table_if_exists_query)
        execute_query(conn_db, create_query)
        
        # Step 2: Insert edge trees into powerline table 
        with open(SKB_file_in, 'r') as src_file:
            for file_line in src_file:
                l_split = file_line.split(' ')
                x = float(l_split[0])
                y = float(l_split[1])
                z = float(l_split[2])
                dz = float(l_split[3])
                mz = z - dz
                
                #TODO potentially change to 5845, also check if SRID is needed on table level as well
                insert_query = sql.SQL("""
                    INSERT INTO {table_name} (x, y, z, dz, mz, shape)
                    VALUES (%s, %s, %s, %s, %s, ST_GeomFromText(%s, 3006))
                """).format(table_name=sql.Identifier(table_name))

                pointz_string = f"POINTZ({x} {y} {mz})"
                tree_data = (x, y, z, dz, mz, pointz_string)
                execute_query(conn_db, insert_query, tree_data)

    except psycopg.Error as e:
        print("Error while working with PostgreSQL:", e)   

try: 
    conn_db = connect_to_database(DB_NAME)
    powerlines_df = pd.read_csv(powerline_list, sep="\t", header=0)

    if conn_db:
        powerlines_df.apply(create_SKB_XYmZ_table, axis=1)
    
    print("Successfully inserted edge trees to PostGIS database.") 

finally: 
    if conn_db is not None:
        conn_db.close()
        print(f"Connection to database {DB_NAME} closed.")

Successfully inserted edge trees to PostGIS database.
Connection to database test_postgis_db closed.


## Conduct distance calculations

In [7]:
import psycopg
from pathlib import Path
import pandas as pd

def calculate_distances(row, conn_db):
    #TODO se över SQL query om de kan förbättras, optimeras
    #TODO Borde ta bort columner först om de existerar 
    LG = row["LG"]
    line = row["line"]
    table_name = f"{LG.lower()}_{line}_skb_xymz"
    wire_table_name = f"{LG.lower()}_{line}_fas"
    
    add_columns_query = sql.SQL("""ALTER TABLE {table_name}
        ADD COLUMN IF NOT EXISTS avst_mz_fas DOUBLE PRECISION,
        ADD COLUMN IF NOT EXISTS avst_hori DOUBLE PRECISION, 
        ADD COLUMN IF NOT EXISTS avst_fas DOUBLE PRECISION;""").format(table_name=sql.Identifier(table_name))
    
    avst_mz_fas_query = sql.SQL("""UPDATE {table_name} p
        SET avst_mz_fas = subquery.distance
        FROM (
            SELECT DISTINCT ON (p.objectid)  
                p.objectid,
                ST_3DDistance(p.shape, l.shape) AS distance
            FROM 
                {table_name} p, 
                {wire_table_name} l
            WHERE 
                ST_3DDWithin(p.shape, l.shape, 100)
            ORDER BY 
                p.objectid, distance ASC
        ) AS subquery
        WHERE p.objectid = subquery.objectid;""").format(table_name=sql.Identifier(table_name), wire_table_name=sql.Identifier(wire_table_name))

    avst_fas_query = sql.SQL("""UPDATE {table_name} p
        SET avst_fas = subquery.distance
        FROM (
            SELECT objectid, avst_mz_fas - dz AS distance
            FROM {table_name}
        ) AS subquery
        WHERE p.objectid = subquery.objectid;""").format(table_name=sql.Identifier(table_name))

    avst_hori_query = sql.SQL("""UPDATE {table_name} p
        SET avst_hori = subquery.distance
        FROM (
            SELECT DISTINCT ON (p.objectid)  
                p.objectid,
                ST_Distance(p.shape, l.shape) AS distance
            FROM 
                {table_name} p, 
                {wire_table_name} l
            WHERE 
                ST_DWithin(p.shape, l.shape, 100)
            ORDER BY 
                p.objectid, distance ASC
        ) AS subquery
        WHERE p.objectid = subquery.objectid;""").format(table_name=sql.Identifier(table_name), wire_table_name=sql.Identifier(wire_table_name))

    # Step 1: Add attributes
    execute_query(conn_db, add_columns_query)
    print(f"Columns were added successfully to table {table_name}")

    # Step 2: Calculate shortest distance between mz and phase.
    execute_query(conn_db, avst_mz_fas_query)
    print(f"avst_mz_fas was calculated succesfully for {LG}_{line}")

    # Step 3: Calculate the shortest distance between tree top (z) and phase with event of potential fall.
    execute_query(conn_db, avst_fas_query)
    print(f"avst_fas was calculated succesfully for {LG}_{line}")

    # Step 4: Calculate the horizontal distance between tree and phase.
    execute_query(conn_db, avst_hori_query)
    print(f"avst_hori was calculated succesfully for {LG}_{line}") 

try: 
    powerlines_df = pd.read_csv(powerline_list, sep="\t", header=0)
    conn_db = connect_to_database(DB_NAME)

    if conn_db:
        powerlines_df.apply(lambda row: calculate_distances(row, conn_db), axis=1)
    
    print("Successfully calculated distances for all wires.") #TODO printed on error, change this
except psycopg.Error as e:
    print("Error while working with PostgreSQL:", e) 
except Exception as e:
    print("Error: ", e)
finally: 
    if conn_db is not None:
        conn_db.close()
        print(f"Connection to database {DB_NAME} closed.")

Columns were added successfully to table lg001_1_skb_xymz
avst_mz_fas was calculated succesfully for LG001_1
avst_fas was calculated succesfully for LG001_1
avst_hori was calculated succesfully for LG001_1
Successfully calculated distances for all wires.
Connection to database test_postgis_db closed.


# Add Littera, Ursprung, Ursprung_Datum, Registreringsdatum

In [8]:
import psycopg
import pandas as pd
from psycopg import sql
from datetime import date

def add_data(row):
    LG = row["LG"]
    line = row["line"]
    littera = row["Littera"]
    regdatum = skanningsdatum[f"{LG}_{line}.0"]   
    table_name = f"{LG.lower()}_{line}_skb_xymz"

    #TODO ledningsgata, insamlingsmetod, matosakerhet x2 should be updated to domains, using foreign keys 
    add_columns_query = sql.SQL("""ALTER TABLE {table_name}
                                    ADD COLUMN IF NOT EXISTS rgdtm DATE, 
                                    ADD COLUMN IF NOT EXISTS ledningsgata FLOAT,
                                    ADD COLUMN IF NOT EXISTS insamlingsmetod FLOAT,
                                    ADD COLUMN IF NOT EXISTS matosakerhet_plan FLOAT,
                                    ADD COLUMN IF NOT EXISTS matosakerhet_hojd FLOAT,
                                    ADD COLUMN IF NOT EXISTS littera VARCHAR(255),
                                    ADD COLUMN IF NOT EXISTS ursprung VARCHAR(255),
                                    ADD COLUMN IF NOT EXISTS ursprung_datum DATE;""").format(table_name=sql.Identifier(table_name))

    # TODO Update littera to work for domain
    # TODO Investigate if % or placeholder should be used
    update_query = sql.SQL("""UPDATE {table_name}
                                SET rgdtm = {regdatum}, 
                                littera = {littera},
                                ursprung = {ursprung},
                                ursprung_datum = {lev_dat};""").format(table_name=sql.Identifier(table_name), 
                                                                       regdatum=sql.Placeholder("regdatum"), 
                                                                       littera=sql.Placeholder("littera"), 
                                                                       ursprung=sql.Placeholder("ursprung"), 
                                                                       lev_dat=sql.Placeholder("lev_dat"))
    updated_values = {"regdatum": regdatum, "littera": littera, "ursprung": urspr, "lev_dat": lev_dat}

    try:
        # Step 1: Add columns
        execute_query(conn_db, add_columns_query)
        print(f"Columns were added successfully to table {table_name}")
        
        # Step 2: Update Littera, Ursprung, Ursprung_Datum, Registreringsdatum 
        execute_query(conn_db, update_query, updated_values)
        print(f"Values were updated successfully in table {table_name}")

    except psycopg.Error as e:
        print("Error while working with PostgreSQL:", e)   

try: 
    conn_db = connect_to_database(DB_NAME)
    powerlines_df = pd.read_csv(powerline_list, sep="\t", header=0)
    df_skanningsdatum = pd.read_csv(scandate_file, sep="\t", header=0) #TODO english variables?
    skanningsdatum = {f"{row['LG']}_{row['line']}": row['skanningsdatum'] for _, row in df_skanningsdatum.iterrows()}
    urspr = "SWECO"
    lev_dat = str(date.today())

    if conn_db:
        powerlines_df.apply(add_data, axis=1) #TODO don't have to pass conn_db on the other places, do like this instead
    
    print("Successfully inserted edge trees to PostGIS database.") 
except Exception as e:
    print("Error: ", e)
finally: 
    if conn_db is not None:
        conn_db.close()
        print(f"Connection to database {DB_NAME} closed.")

Columns were added successfully to table lg001_1_skb_xymz
Values were updated successfully in table lg001_1_skb_xymz
Successfully inserted edge trees to PostGIS database.
Connection to database test_postgis_db closed.


## Merge edge trees for all powerlines into one table

In [23]:
def list_skb_xymz_tables():
    select_tables_query = sql.SQL("""
        SELECT table_name
        FROM information_schema.tables
        WHERE table_name LIKE '%%skb_xymz' AND table_schema = 'public';
    """)

    result = execute_query(conn_db, select_tables_query, fetch=True)
    table_names = [t[0] for t in result]

    return table_names

def merge_tables(table_names):
    table_name = 'kantträd_oklippta'

    create_table_query = sql.SQL("""
        CREATE TABLE IF NOT EXISTS {table_name} AS
        TABLE {first_table} WITH NO DATA;
    """).format(table_name=sql.Identifier(table_name), first_table=sql.Identifier(table_names[0]))

    execute_query(conn_db, create_table_query)

    for table in table_names:
        insert_query = sql.SQL("""
            INSERT INTO {table_name}
            SELECT * FROM {table}
        """).format(table_name=sql.Identifier(table_name), table=sql.Identifier(table))

        execute_query(conn_db, insert_query)

try: 
    conn_db = connect_to_database(DB_NAME)

    if conn_db:
        table_names = list_skb_xymz_tables()
        if table_names:
            print(f"Found tables: {table_names}")
            merge_tables(table_names)
            print("Successfully merged all edge trees tables into one table.") 
        else:
            print("No tables ending with 'skb_xymz' found.")
except Exception as e:
    print("Error: ", e)
finally: 
    if conn_db is not None:
        conn_db.close()
        print(f"Connection to database {DB_NAME} closed.")

Found tables: ['lg001_1_skb_xymz', 'lg001_2_skb_xymz', 'lg001_3_skb_xymz']
Successfully merged all edge trees tables into one table.
Connection to database test_postgis_db closed.


## Convert edge tree PostGIS table to file geodatabase

In [ ]:
# TODO Rename SKB_XYmZ feature class, dataset and table from the start to SKB_XYZ
# TODO Delete fields like x, y, z
# TODO check if shape should be mz or z, currently mz 

import os 
import pandas as pd
import subprocess

def postGIS_to_gdb(row):
    LG = row["LG"]
    line = row["line"]    
    table_name = f"{LG}_{line}_SKB_XYmZ"
    feature_dataset_name = "SKB_XYmZ"

    ogr_command = [
        ogr2ogr_path,
        "-f", "OpenFileGDB",
        working_gdb,      
        pg_connection,
        "-a_srs", "EPSG:3006",
        "-nlt", "POINTZ",
        "-dim", "3",
        "-overwrite", #TODO ändra detta när det blir flera dgn-filer, t.ex. till append
        "-sql", f"SELECT * FROM public.{table_name}",
        "-nln", table_name,
        "-lco", f"FEATURE_DATASET={feature_dataset_name}"
    ]

    try:
        subprocess.run(ogr_command, check=True, capture_output=True, text=True)
        print(f"Successfully converted PostGIS {table_name} into gdb {working_gdb}.")
    except subprocess.CalledProcessError as e:
        print(f"Error during import: {e}")

try:
    pg_connection = f"PG:host={HOST} dbname={DB_NAME} user={USER} password={PASSWORD} port={PORT}"
    ogr2ogr_path = ogr2ogr_path
    os.environ["PROJ_LIB"] = proj_lib_path #TODO potentially solve this in another way, but needed to specify EPSG:3006
    os.environ["GDAL_DATA"] = gdal_data_path

    powerlines_df = pd.read_csv(powerline_list, sep="\t", header=0)
    powerlines_df.apply(postGIS_to_gdb, axis=1)
except Exception as e:
    print("Error: ", e)

Successfully converted PostGIS LG001_1_SKB_XYmZ into gdb C:/Users/SE1K4H/Desktop/SVK-Analys-Filer/pythonkörningar/test_postGIS\working_250201_PostGIS.gdb.


# Slå ihop kantträd för alla ledningar till en featureclass

In [ ]:
#working_gdb = "C:\SVK_2024\python_work_dir\working_250218.gdb"
print(working_gdb)
#print(Path(working_gdb,"SKB_XYZ"))
arcpy.env.workspace = working_gdb
#arcpy.env.workspace = os.path.join(working_gdb,"SKB_XYZ")
print(arcpy.env.workspace)
feature_classes = arcpy.ListFeatureClasses(feature_dataset="SKB_XYZ")
print(feature_classes)
output_feature_classes = os.path.join(working_gdb,"kantträd_oklippta") #"c:\workspace\working_25.gdb\merged_feature_class"
print(output_feature_classes)
arcpy.Merge_management(feature_classes, output_feature_classes)
print("körning av cell klar")

# Klipp träffpunkter
Se till att använda aktuella data i LG_polygons och station_polygons

In [ ]:
import arcpy
import os
import pandas as pd

#fc_ledningsgator = r"Q:\Projekt\Data_2024\Underlag_SVK\Bestallningsunderlag_2024.gdb\GNG_LEDNINGSGATA" #os.path.join(gdb, "Gator2021")
#fc_stationsomraden = r"Q:\Projekt\Data_2024\Underlag_SVK\Bestallningsunderlag_2024.gdb\GNG_STATIONSOMRADE" #os.path.join(gdb, "STATIONSOMRADE")

arcpy.env.overwriteOutput = True

##fd_ej_akuta = os.path.join(gdb, "ej_akuta")
#fd_all = os.path.join(gdb, "all")
#fd_traffpunkter = os.path.join(gdb, "traffpunkter")

#fc_all = os.path.join(fd_all, f"all_{version}")
#fc_traffpunkter = os.path.join(fd_traffpunkter, f"traffpunkter_{version}")
tmp_layer_traffpunkter = os.path.join(working_gdb, "tmp_layer_traffpunkter")
print(f"tmp_layer_traffpunkter: {tmp_layer_traffpunkter}")

fc_traffpunkter = os.path.join(results_gdb, 'kantträd')
print(f"fc_traffpunkter: {fc_traffpunkter}")

fc_all = os.path.join(working_gdb, "kantträd_oklippta")
print(f"fc_all: {fc_all}")

#arcpy.management.Delete(tmp_layer_traffpunkter) #behövs detta bara i fall man kör om funktionen och tmp_layer_traffpunkter existerar 



# Gör kopia av oklippta
arcpy.management.CopyFeatures(fc_all, fc_traffpunkter)

arcpy.management.MakeFeatureLayer(fc_traffpunkter, tmp_layer_traffpunkter)

# Steg 1
# Inom fastbredd och stationsområden: ta bort alla inom 7 m horisontellt avstånd från fas
#
# 1.1 Välj alla inom fastbredd
arcpy.management.SelectLayerByLocation(in_layer = tmp_layer_traffpunkter, 
                                       overlap_type = "INTERSECT", 
                                       select_features = LG_polygons,
                                       selection_type = "NEW_SELECTION")
# 1.2 Lägg till alla inom stationsområden
arcpy.management.SelectLayerByLocation(in_layer = tmp_layer_traffpunkter, 
                                       overlap_type = "INTERSECT", 
                                       select_features = station_polygons,
                                       selection_type = "ADD_TO_SELECTION")
# 1.3 Av de valda, behåll punkter med avst_hori < 7 m
arcpy.management.SelectLayerByAttribute(in_layer_or_view = tmp_layer_traffpunkter,
                                        selection_type = "SUBSET_SELECTION", 
                                        where_clause = '"AVSTAND_HORISONTELLT" < 7')
# 1.4 Radera valda
arcpy.management.DeleteFeatures(tmp_layer_traffpunkter)

# Steg 2
# Inom fastbredd och stationsområden: ta bort alla med trädhöjd <= 4.5 m och avst_fas >= 5 m
#
# 2.1 Välj alla inom fastbredd
arcpy.management.SelectLayerByLocation(in_layer = tmp_layer_traffpunkter, 
                                       overlap_type = "INTERSECT", 
                                       select_features = LG_polygons,
                                       selection_type = "NEW_SELECTION")
# 2.2 Lägg till alla inom stationsområden
arcpy.management.SelectLayerByLocation(in_layer = tmp_layer_traffpunkter, 
                                       overlap_type = "INTERSECT", 
                                       select_features = station_polygons,
                                       selection_type = "ADD_TO_SELECTION")
# 2.3 Av de valda, gör gör subset selection med punkter med trädhöjd <= 4.5 OCH avst_fas >= 5
arcpy.management.SelectLayerByAttribute(in_layer_or_view = tmp_layer_traffpunkter,
                                       selection_type = "SUBSET_SELECTION",
                                       where_clause = '"DELTA_HOJD" <= 4.5 AND "AVSTAND_FAS" >= 5')
# 2.4 Radera valda
arcpy.management.DeleteFeatures(tmp_layer_traffpunkter)

## Kopiera kvarvarande punkter till traffpunkter
#arcpy.management.CopyFeatures(tmp_layer_traffpunkter, fc_traffpunkter) # Överflödigt?

# Ta bort det temporära lagret
arcpy.management.Delete(tmp_layer_traffpunkter)

print("körning av cell klar")

In [ ]:
import os
import pandas as pd
from pathlib import Path
import datetime
from datetime import date
import arcpy

#r"C:\SVK_2024\python_work_dir\results_250218.gdb"
#results_gdb = os.path.join(local_dir, f"results_{run_ID}.gdb")
#print(Path(results_gdb))

arcpy.env.workspace = working_gdb
print(arcpy.env.workspace)
kanttrad = os.path.join(results_gdb, 'kantträd')
print(kanttrad)

arcpy.management.AssignDomainToField(kanttrad, "LEDNINGSGATA", "cvd_LEDNINGSGATA")
arcpy.management.AssignDomainToField(kanttrad, "LITTERA", "cvd_LITTERA_LEDNING")
arcpy.management.AssignDomainToField(kanttrad, "Matosakerhet_Hojd", "cvd_MATOSAKERHET")
arcpy.management.AssignDomainToField(kanttrad, "Matosakerhet_Plan", "cvd_MATOSAKERHET")
arcpy.management.AssignDomainToField(kanttrad, "Insamlingsmetod", "cvd_INMATNINGSMETOD")



#print(kanttrad)
#arcpy.env.workspace = kanttrad
#arcpy.env.workspace(kanttrad)
#print(arcpy.env.workspace)
#feature_classes = arcpy.ListFeatureClasses(feature_dataset="SKB_XYZ")
print("körning av cell klar")


# Kontroll
### Här kan man lägga in egna skript för kontroll av resultatet

In [ ]:
# FLytta till RBX-notebook.
# Listar alla LG - littera som finns i RBX-resultatet
# Jämför detta med logg-filen för att se att rätt ledningar
# är med i resultatet (inte alla ledningar har RBX-punkter)

import arcpy
import os
import geopandas as gpd

gdb_path = r"C:\SVK_2024\pythonkörningar\results_2501xx.gdb"
feature_class = "RBX_closest_points"

gdf = gpd.read_file(gdb_path, layer=feature_class)

gdf = gdf[["LEDNINGSGATA", "LITTERA"]]
unique_df = gdf.drop_duplicates().sort_values(["LEDNINGSGATA", "LITTERA"])

print(unique_df)